# 如何使用少样本示例

:::info 先决条件

本指南假定您熟悉以下概念：
- [Prompt templates](/docs/concepts/prompt_templates)
- [Example selectors](/docs/concepts/example_selectors)
- [LLMs](/docs/concepts/text_llms)
- [Vectorstores](/docs/concepts/vectorstores)

:::

在本指南中，我们将学习如何创建一个简单的 prompt template，在生成时为模型提供示例输入和输出。为 LLM 提供一些这样的示例称为 [少样本（few-shotting）](/docs/concepts/few_shot_prompting/)，这是一种简单而强大的引导生成的方式，在某些情况下可以显著提高模型性能。

少样本 prompt template 可以由一组示例构建，也可以由一个负责从已定义集合中选择示例子集的 [Example Selector](https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.base.BaseExampleSelector.html) 类构建。

本指南将介绍如何使用字符串 prompt template 进行少样本。有关使用聊天消息进行少样本的指南，请参阅 [此处](/docs/how_to/few_shot_examples_chat/)。

## 创建一个用于少样本示例的格式化器

配置一个格式化器，它会将少样本示例格式化为字符串。此格式化器应为 `PromptTemplate` 对象。

In [1]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

## 创建示例集

接下来，我们将创建一个少样本示例列表。每个示例都应该是代表我们上面定义的格式化程序提示的示例输入的字典。

In [2]:
examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

让我们用一个例子来测试格式化提示：

In [13]:
print(example_prompt.invoke(examples[0]).to_string())

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



### 将示例和格式化器传递给 `FewShotPromptTemplate`

最后，创建一个 [`FewShotPromptTemplate`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html) 对象。此对象接收少样本示例和少样本示例的格式化器。当此 `FewShotPromptTemplate` 被格式化时，它会使用 `example_prompt` 来格式化传入的示例，然后将它们添加到最终提示中，放在 `suffix` 之前：

In [14]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

通过向模型提供此类示例，我们可以引导模型获得更好的响应。

## 使用示例选择器

我们将复用上一节中的示例集和格式化器。但是，我们不会直接将示例馈送给 `FewShotPromptTemplate` 对象，而是将它们馈送给一个名为 `SemanticSimilarityExampleSelector` 的 `ExampleSelector` 实现实例。这个类根据输入与初始集合中示例的相似度来选择少样本示例。它使用一个嵌入模型来计算输入与少样本示例之间的相似度，并使用一个向量存储来执行最近邻搜索。

为了展示它的作用，让我们初始化一个实例并单独调用它：

In [4]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1,
)

# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Who was the father of Mary Ball Washington?


answer: 
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

question: Who was the maternal grandfather of George Washington?


现在，我们来创建一个 `FewShotPromptTemplate` 对象。该对象接收示例选择器和用于少样本示例的格式化提示。

In [5]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Who was the father of Mary Ball Washington?


## 后续步骤

你现在已经学会了如何在提示中添加少样本示例。

接下来，请查看本节中有关提示模板的其他操作指南、关于使用聊天模型进行少样本的[相关操作指南](/docs/how_to/few_shot_examples_chat)，或其他的[示例选择器操作指南](/docs/how_to/example_selectors/)。